In [1]:
import numpy as np

class SimpleRNN:
    def __init__(self, input_size, hidden_size, output_size):
        self.hidden_size = hidden_size

        # Weights initialization
        self.Wxh = np.random.randn(hidden_size, input_size) * 0.01  # input to hidden
        self.Whh = np.random.randn(hidden_size, hidden_size) * 0.01  # hidden to hidden
        self.Why = np.random.randn(output_size, hidden_size) * 0.01  # hidden to output

        # Biases initialization
        self.bh = np.zeros((hidden_size, 1))  # hidden layer bias
        self.by = np.zeros((output_size, 1))  # output layer bias

    def forward(self, inputs):
        h = np.zeros((self.hidden_size, 1))  # Initial hidden state
        self.h_states = [h] # Store initial hidden state for backpropagation

        for x in inputs:
            x = x.reshape(-1, 1)  # Reshape input to column vector
            h = np.tanh(np.dot(self.Wxh, x) + np.dot(self.Whh, h) + self.bh)  # Update hidden state
            self.h_states.append(h)  # Store hidden state

        y = np.dot(self.Why, h) + self.by  # Output layer
        return y

    def loss(self, y_pred, y_true):
        return np.mean((y_pred - y_true) ** 2)  # Mean Squared Error

    def backward(self, inputs, y_pred, y_true, learning_rate=0.01):
        # Calculate loss derivative
        dy = y_pred - y_true
        dWhy = np.dot(dy, self.h_states[-1].T)
        dby = dy

        # Backpropagation through time
        dh = np.dot(self.Why.T, dy)
        dWxh, dWhh, dbh = 0, 0, 0 # Initialize gradient accumulators

        for t in reversed(range(len(inputs))):

            x = inputs[t].reshape(-1, 1) # Current input

            # Using previous hidden state for gradient calculation
            prev_h = self.h_states[t]

            dh_raw = (1 - self.h_states[t+1] ** 2) * dh  # Derivative of tanh w.r.t. the current hidden state

            # sum up gradients
            dWxh += np.dot(dh_raw, x.T)
            dWhh += np.dot(dh_raw, prev_h.T)
            dbh += dh_raw

            # Backpropagate the gradient to the previous time step
            dh = np.dot(self.Whh.T, dh_raw)

        # Update weights and biases
        self.Wxh -= learning_rate * dWxh
        self.Whh -= learning_rate * dWhh
        self.Why -= learning_rate * dWhy
        self.bh -= learning_rate * dbh
        self.by -= learning_rate * dby

        return dWhy, dby

# Example usage
if __name__ == "__main__":
    rnn = SimpleRNN(input_size=3, hidden_size=5, output_size=2)
    inputs = np.random.rand(10, 3)
    target = np.random.rand(2, 1)    # Target output

    # Forward pass
    y_pred = rnn.forward(inputs)
    print("Prediction:", y_pred)

    # Calculate loss
    loss = rnn.loss(y_pred, target)
    print("Loss:", loss)

    # Backward pass
    rnn.backward(inputs, y_pred, target)



Prediction: [[-5.49295067e-05]
 [ 2.77178465e-04]]
Loss: 0.052589565739662246
